In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score,mean_squared_error

In [2]:
train_df = pd.read_csv("train (2).csv")
test_df = pd.read_csv("test (2).csv")

In [3]:
label = train_df["SalePrice"]
train_df.drop("SalePrice",axis=1,inplace=True)
train_df.drop("Id",inplace=True,axis=1)
test_df.drop("Id",inplace=True,axis=1)

In [4]:
# check the numbers of categorical features in train_df

cat_cols = train_df.select_dtypes(include='object').columns.tolist()
print(f"Number of categorical features: {len(cat_cols)}")

# prompt: to chech the numbers of numeric features in train_df

numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(f"Number of numeric features: {len(numeric_cols)}")

Number of categorical features: 43
Number of numeric features: 36


In [5]:
combined_df = pd.concat([train_df, test_df], axis=0)

In [6]:
def cat_num(data):
    # check the numbers of categorical features in train_df

    cat_cols = data.select_dtypes(include='object').columns.tolist()
    print(f"Number of categorical features: {len(cat_cols)}")
    cat_col_indices = list(data.columns.get_loc(col) for col in cat_cols)

    # prompt: to chech the numbers of numeric features in train_df

    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    print(f"Number of numeric features: {len(numeric_cols)}")
    numeric_cols_indices = list(data.columns.get_loc(col) for col in numeric_cols)
    
  
    return cat_col_indices, numeric_cols_indices

In [7]:
cat_col_indices, numeric_cols_indices = cat_num(combined_df)

Number of categorical features: 43
Number of numeric features: 36


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
cat_col_indices_values_df = pd.DataFrame(combined_df.iloc[:, cat_col_indices].values, columns=combined_df.iloc[:, cat_col_indices].columns)
cat_col_indices_values_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [10]:
numeric_cols_indices_values_df = pd.DataFrame(combined_df.iloc[:, numeric_cols_indices].values, columns=combined_df.iloc[:, numeric_cols_indices].columns)
numeric_cols_indices_values_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0


In [11]:
kill = []
for i in cat_col_indices_values_df.columns:
    kill.append(list(cat_col_indices_values_df[i].unique()))

print(kill)

[['RL', 'RM', 'C (all)', 'FV', 'RH', nan], ['Pave', 'Grvl'], [nan, 'Grvl', 'Pave'], ['Reg', 'IR1', 'IR2', 'IR3'], ['Lvl', 'Bnk', 'Low', 'HLS'], ['AllPub', 'NoSeWa', nan], ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'], ['Gtl', 'Mod', 'Sev'], ['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst', 'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes', 'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert', 'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU', 'Blueste'], ['Norm', 'Feedr', 'PosN', 'Artery', 'RRAe', 'RRNn', 'RRAn', 'PosA', 'RRNe'], ['Norm', 'Artery', 'RRNn', 'Feedr', 'PosN', 'PosA', 'RRAn', 'RRAe'], ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], ['2Story', '1Story', '1.5Fin', '1.5Unf', 'SFoyer', 'SLvl', '2.5Unf', '2.5Fin'], ['Gable', 'Hip', 'Gambrel', 'Mansard', 'Flat', 'Shed'], ['CompShg', 'WdShngl', 'Metal', 'WdShake', 'Membran', 'Tar&Grv', 'Roll', 'ClyTile'], ['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing', 'CemntBd

In [12]:
counter = 0
for i in kill:
    one = len(i)
    # print(one)
    counter += one
    
print(counter)

274


In [13]:
feature_map = {}
idx = 0

# Iterate over each list of categories
for category_list in kill:
# Iterate over each category in the list
    for category in category_list:
        # Assign unique index to the category and update the index
        feature_map[category] = idx
        idx += 1

print(feature_map)


{'RL': 0, 'RM': 1, 'C (all)': 2, 'FV': 3, 'RH': 4, nan: 267, 'Pave': 10, 'Grvl': 9, 'Reg': 11, 'IR1': 12, 'IR2': 13, 'IR3': 14, 'Lvl': 15, 'Bnk': 16, 'Low': 17, 'HLS': 18, 'AllPub': 19, 'NoSeWa': 20, 'Inside': 22, 'FR2': 23, 'Corner': 24, 'CulDSac': 25, 'FR3': 26, 'Gtl': 27, 'Mod': 208, 'Sev': 210, 'CollgCr': 30, 'Veenker': 31, 'Crawfor': 32, 'NoRidge': 33, 'Mitchel': 34, 'Somerst': 35, 'NWAmes': 36, 'OldTown': 37, 'BrkSide': 38, 'Sawyer': 39, 'NridgHt': 40, 'NAmes': 41, 'SawyerW': 42, 'IDOTRR': 43, 'MeadowV': 44, 'Edwards': 45, 'Timber': 46, 'Gilbert': 47, 'StoneBr': 48, 'ClearCr': 49, 'NPkVill': 50, 'Blmngtn': 51, 'BrDale': 52, 'SWISU': 53, 'Blueste': 54, 'Norm': 64, 'Feedr': 67, 'PosN': 68, 'Artery': 65, 'RRAe': 71, 'RRNn': 66, 'RRAn': 70, 'PosA': 69, 'RRNe': 63, '1Fam': 72, '2fmCon': 73, 'Duplex': 74, 'TwnhsE': 75, 'Twnhs': 76, '2Story': 77, '1Story': 78, '1.5Fin': 79, '1.5Unf': 80, 'SFoyer': 81, 'SLvl': 82, '2.5Unf': 83, '2.5Fin': 84, 'Gable': 85, 'Hip': 86, 'Gambrel': 87, 'Mansar

In [14]:
# Map the values in train_df to numerical indices using feature_map
combined_df = combined_df.applymap(lambda x: feature_map.get(x, x))

print(combined_df.head())

   MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  \
0          60         0         65.0     8450      10    267        11   
1          20         0         80.0     9600      10    267        11   
2          60         0         68.0    11250      10    267        12   
3          70         0         60.0     9550      10    267        12   
4          60         0         84.0    14260      10    267        12   

   LandContour  Utilities  LotConfig  ...  ScreenPorch  PoolArea  PoolQC  \
0           15         19         22  ...            0         0     267   
1           15         19         23  ...            0         0     267   
2           15         19         22  ...            0         0     267   
3           15         19         24  ...            0         0     267   
4           15         19         23  ...            0         0     267   

   Fence  MiscFeature  MiscVal  MoSold  YrSold  SaleType  SaleCondition  
0    267          267   

In [15]:
# import torch.nn.functional as F

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attention = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None):
        src2, _ = self.self_attention(src, src, src, attn_mask=src_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.ffn(src)
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, n_heads, d_ff, dropout=0.1):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(num_layers)
        ])

    def forward(self, src, src_mask=None):
        for layer in self.layers:
            src = layer(src, src_mask)
        return src

In [16]:
# Example usage:
# Define parameters
num_layers = 6
d_model = len(cat_cols)
n_heads = 1
d_ff = 2048
dropout = 0.5
src_seq_len = len(cat_cols)
batch_size = 32
vocab_size = counter

In [17]:
torch.manual_seed(42)

# Create encoder instance
encoder = Encoder(num_layers, d_model, n_heads, d_ff, dropout)

# Example input tensor
# src_input = torch.randn(batch_size, src_seq_len, d_model)

In [18]:
cat_col_indices_values_df.values[0]

array(['RL', 'Pave', nan, 'Reg', 'Lvl', 'AllPub', 'Inside', 'Gtl',
       'CollgCr', 'Norm', 'Norm', '1Fam', '2Story', 'Gable', 'CompShg',
       'VinylSd', 'VinylSd', 'BrkFace', 'Gd', 'TA', 'PConc', 'Gd', 'TA',
       'No', 'GLQ', 'Unf', 'GasA', 'Ex', 'Y', 'SBrkr', 'Gd', 'Typ', nan,
       'Attchd', 'RFn', 'TA', 'TA', 'Y', nan, nan, nan, 'WD', 'Normal'],
      dtype=object)

In [19]:
# torch.manual_seed(42)
# # Example input tensor
# src_input = cat_col_indices_values_df.values
# src_input = torch.Tensor(src_input)
# print("Source Input",src_input)
# # enc=encoder(src_input)

In [20]:
def min_max_scaling(data):
    min_val = data.min()
    max_val = data.max()
    scaled_data = (data - min_val) / (max_val - min_val)
    return scaled_data


In [21]:
def z_score_scaling(data):
    mean = data.mean()
    std_dev = data.std()
    scaled_data = (data - mean) / std_dev
    return scaled_data


In [22]:
import numpy as np

In [23]:
val = np.nan
val

nan

In [24]:
def robust_scaling(data):

    median = np.median(data, axis=0)
    q1 = np.percentile(data, 25, axis=0)
    q3 = np.percentile(data, 75, axis=0)
    iqr = q3 - q1
    scaled_data = (data - median) / iqr
    val = np.nan
    for i in data:
        if i == val:
            i = 0
    return scaled_data

In [25]:
for i in combined_df:
    if combined_df[i].isnull().sum() == 0:
        continue
    else:
        combined_df[i].fillna((combined_df[i].mean()), inplace=True)

In [26]:
def preprocess(train_df,cat_col_indices=cat_col_indices,
               numeric_cols_indices=numeric_cols_indices,
               scaling=z_score_scaling,
               feature_map=feature_map,
               encoder=encoder,random_seed=42):
    
    cat_col_indices_values_df = pd.DataFrame(train_df.iloc[:, cat_col_indices].values, columns=train_df.iloc[:, cat_col_indices].columns)
    # print("Categorical features\n",cat_col_indices_values_df.head())
    numeric_cols_indices_values_df = pd.DataFrame(train_df.iloc[:, numeric_cols_indices].values, columns=train_df.iloc[:, numeric_cols_indices].columns)
    # print("Continouns Variable",numeric_cols_indices_values_df.head())
    
    # Map the values in train_df to numerical indices using feature_map
    mapped_cat_col_indices_values_df = cat_col_indices_values_df.applymap(lambda x: feature_map.get(x, x))
    

    # print("Mapped Cat_col",mapped_cat_col_indices_values_df.head())
   
    # Example input tensor
    src_input = mapped_cat_col_indices_values_df.values
    src_input = torch.Tensor(src_input)
    # print("Source Input",src_input)
    
    torch.manual_seed(42)
    new = []
    enc_values = []
    
    for i in np.arange(len(src_input)):
        new.append(encoder(src_input[i].unsqueeze(dim=0)))
        

    for i in new:
        enc_values.append(i.squeeze().detach().numpy())
        
    enc_values = pd.DataFrame(enc_values)
    enc_values.head()
    # print(enc_values[:5])
    enc_values.columns = cat_col_indices_values_df.columns
        
    if scaling == min_max_scaling:
        scale = numeric_cols_indices_values_df.apply(min_max_scaling)
        # print("Categorical Value Scaling",scale.head())
        
    elif scaling == z_score_scaling:
        scale = numeric_cols_indices_values_df.apply(z_score_scaling)
        # print("Numerical Value scaling",scale.head())
        
    elif scaling == robust_scaling:
        scale = numeric_cols_indices_values_df.apply(robust_scaling)
    
    else:
         scale = numeric_cols_indices_values_df.apply(scale)
        
    df = pd.concat([enc_values,scale],axis=1)
    
    return df

In [27]:
import numpy as np

In [28]:
data = preprocess(combined_df,scaling = min_max_scaling)
data.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.724521,-1.343566,2.594772,-0.136496,-0.279457,2.300155,-2.335210,-1.022328,-1.693199,-1.041042,...,0.368280,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50
1,-1.380468,0.237319,-0.380779,-2.800719,0.417525,0.621277,-1.444886,1.055115,-0.652438,-0.154129,...,0.309140,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25
2,2.566792,-0.078292,-0.289818,-0.504459,0.224014,-0.838462,0.455274,-0.161552,-0.005458,-0.350333,...,0.408602,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50
3,-0.496961,0.501868,-1.010074,-0.455587,0.257685,-0.502064,2.494398,-0.296982,0.763023,-1.010265,...,0.431452,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00
4,0.721316,-0.194268,-1.348781,-0.882696,-1.058455,1.649538,2.311634,-0.563940,-0.949086,-0.963013,...,0.561828,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50


In [29]:
train_df = data[:train_df.shape[0]]

In [30]:
train_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.724521,-1.343566,2.594772,-0.136496,-0.279457,2.300155,-2.335210,-1.022328,-1.693199,-1.041042,...,0.368280,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50
1,-1.380468,0.237319,-0.380779,-2.800719,0.417525,0.621277,-1.444886,1.055115,-0.652438,-0.154129,...,0.309140,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25
2,2.566792,-0.078292,-0.289818,-0.504459,0.224014,-0.838462,0.455274,-0.161552,-0.005458,-0.350333,...,0.408602,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50
3,-0.496961,0.501868,-1.010074,-0.455587,0.257685,-0.502064,2.494398,-0.296982,0.763023,-1.010265,...,0.431452,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00
4,0.721316,-0.194268,-1.348781,-0.882696,-1.058455,1.649538,2.311634,-0.563940,-0.949086,-0.963013,...,0.561828,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50


In [31]:
train_df.shape

(1460, 79)

In [32]:
test_df = data[train_df.shape[0]:]

In [33]:
test_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1460,1.558121,1.433659,0.978115,-1.796862,-0.609416,-2.093978,-0.806654,-1.060021,-0.639688,0.584802,...,0.490591,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0
1461,0.722953,0.447020,1.200569,0.044339,0.879514,1.052186,1.088676,-1.611033,-0.755384,-0.717389,...,0.209677,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0
1462,-0.417156,0.364050,0.933918,0.265974,0.904456,0.164326,-0.005205,-3.779995,0.623014,0.621131,...,0.323925,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0
1463,-0.286605,0.909545,-1.044365,-0.370992,1.362625,-0.690128,0.410211,-1.294087,0.105152,-0.313168,...,0.315860,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0
1464,-0.151735,-1.416171,0.329894,0.710135,0.990364,1.400850,-0.089215,-0.483653,-1.641525,-1.166972,...,0.340054,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0


In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_val, y_train, y_val = train_test_split(train_df,label,random_state=42,test_size=0.25)

In [36]:
def run_model(model,X_train=X_train,y_train=y_train,X_test=X_val,y_test=y_val):

    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_r2_score = r2_score(y_train,y_train_pred).round(4)
    model_mse = mean_squared_error(y_train,y_train_pred).round(4)
    model_rmse = np.sqrt(model_mse).round(4)

    model_test_r2_score = r2_score(y_test,y_test_pred).round(4)
    model_test_mse = mean_squared_error(y_test,y_test_pred).round(4)
    model_test_rmse = np.sqrt(model_test_mse).round(4)

    print("Model Performance For Traning Set")
    print("--"*5)
    print("r2_score: ", model_r2_score)
    print("mean squared error: ", model_mse)
    print("rmse: ", model_test_rmse)
    print("--"*5)

    print("Model Performance For Test Set")
    print("--"*5)
    print("r2_score: ", model_test_r2_score)
    print("mean squared error: ", model_test_mse)
    print("rmse: ", model_test_rmse)
    print("--"*5)
    print(model)
    print("__"*5)

    model_name = model.__repr__()

    # Check if the length of the model name is greater than 20
    if len(model_name) > 20:
        # Take the first ten letters of the model name
        model_name = model_name[:10]

    return {"Model Name" : model_name,
            "r2_score" : model_test_r2_score,
            "mean squared error" : model_test_mse,
            "rmse" : model_test_rmse}


In [37]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=200,max_leaf_nodes=60)
# random_forest = run_model(rfr)
# print(random_forest)

In [38]:
from catboost import CatBoostRegressor
cbr = CatBoostRegressor(silent=True)
# cat_boost = run_model(cbr)
# print(cat_boost)

In [39]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

In [40]:
models = {
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'CatBoost': cb.CatBoostRegressor(silent=True)
}

In [41]:
train_df = data[:train_df.shape[0]]
test_df = data[train_df.shape[0]:]

In [42]:
X_train, X_val, y_train, y_val = train_test_split(train_df,label,random_state=42,test_size=0.25)

In [43]:
rf = run_model(rfr)

Model Performance For Traning Set
----------
r2_score:  0.9486
mean squared error:  312265715.4388
rmse:  30698.4219
----------
Model Performance For Test Set
----------
r2_score:  0.8655
mean squared error:  942393105.4663
rmse:  30698.4219
----------
RandomForestRegressor(max_leaf_nodes=60, n_estimators=200)
__________


In [44]:
cb = run_model(cbr)

Model Performance For Traning Set
----------
r2_score:  0.998
mean squared error:  11987082.5943
rmse:  28160.374
----------
Model Performance For Test Set
----------
r2_score:  0.8868
mean squared error:  793006663.2795
rmse:  28160.374
----------
__________


In [45]:
result = []
for name, model in models.items():
    mod = run_model(model)
    # df = pd.DataFrame([run_model(model)])
    result.append(mod)

Model Performance For Traning Set
----------
r2_score:  0.9744
mean squared error:  155194209.6078
rmse:  30318.7574
----------
Model Performance For Test Set
----------
r2_score:  0.8688
mean squared error:  919227049.6039
rmse:  30318.7574
----------
RandomForestRegressor()
__________
Model Performance For Traning Set
----------
r2_score:  0.9674
mean squared error:  197918776.1585
rmse:  29743.9806
----------
Model Performance For Test Set
----------
r2_score:  0.8737
mean squared error:  884704383.605
rmse:  29743.9806
----------
GradientBoostingRegressor()
__________
Model Performance For Traning Set
----------
r2_score:  1.0
mean squared error:  74093.0054
rmse:  30391.7283
----------
Model Performance For Test Set
----------
r2_score:  0.8681
mean squared error:  923657148.7711
rmse:  30391.7283
----------
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_

In [46]:
result

[{'Model Name': 'RandomFore',
  'r2_score': 0.8688,
  'mean squared error': 919227049.6039,
  'rmse': 30318.7574},
 {'Model Name': 'GradientBo',
  'r2_score': 0.8737,
  'mean squared error': 884704383.605,
  'rmse': 29743.9806},
 {'Model Name': 'XGBRegress',
  'r2_score': 0.8681,
  'mean squared error': 923657148.7711,
  'rmse': 30391.7283},
 {'Model Name': 'LGBMRegressor()',
  'r2_score': 0.8807,
  'mean squared error': 835933451.3745,
  'rmse': 28912.5138},
 {'Model Name': '<catboost.',
  'r2_score': 0.8868,
  'mean squared error': 793006663.2795,
  'rmse': 28160.374}]

In [47]:
for i in result:
    print(i)
    columns = i.keys()

{'Model Name': 'RandomFore', 'r2_score': 0.8688, 'mean squared error': 919227049.6039, 'rmse': 30318.7574}
{'Model Name': 'GradientBo', 'r2_score': 0.8737, 'mean squared error': 884704383.605, 'rmse': 29743.9806}
{'Model Name': 'XGBRegress', 'r2_score': 0.8681, 'mean squared error': 923657148.7711, 'rmse': 30391.7283}
{'Model Name': 'LGBMRegressor()', 'r2_score': 0.8807, 'mean squared error': 835933451.3745, 'rmse': 28912.5138}
{'Model Name': '<catboost.', 'r2_score': 0.8868, 'mean squared error': 793006663.2795, 'rmse': 28160.374}


In [48]:
columns

dict_keys(['Model Name', 'r2_score', 'mean squared error', 'rmse'])

In [50]:
import pandas as pd
# Create DataFrame
# data = [result,random_forest]
# print(data)
# robust_scaling_df = pd.DataFrame.from_dict(data, orient='index', columns=columns)

df = pd.DataFrame(result)

# print(robust_scaling_df)
print(df.head())

        Model Name  r2_score  mean squared error        rmse
0       RandomFore    0.8688        9.192270e+08  30318.7574
1       GradientBo    0.8737        8.847044e+08  29743.9806
2       XGBRegress    0.8681        9.236571e+08  30391.7283
3  LGBMRegressor()    0.8807        8.359335e+08  28912.5138
4       <catboost.    0.8868        7.930067e+08  28160.3740


In [51]:
df

,Model Name,r2_score,mean squared error,rmse
0,RandomFore,0.8688,9.192270e+08,30318.7574
1,GradientBo,0.8737,8.847044e+08,29743.9806
2,XGBRegress,0.8681,9.236571e+08,30391.7283
3,LGBMRegressor(),0.8807,8.359335e+08,28912.5138
4,<catboost.,0.8868,7.930067e+08,28160.3740


In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define base models
base_models = [
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    xgb.XGBRegressor(),
    lgb.LGBMRegressor(),
    cb.CatBoostRegressor(silent=True)
]

# Define parameters for grid search
parameters = [
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]},  # RandomForest
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]},  # GradientBoosting
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]},  # XGBoost
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]},  # LightGBM
    {'iterations': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]}  # CatBoost
]

# Train and tune base models
tuned_base_models = []
for model, params in zip(base_models, parameters):
    grid_search = GridSearchCV(model, params, cv=5)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    tuned_model = model.set_params(**best_params)
    tuned_base_models.append(tuned_model)

# Create StackingCVRegressor with Lasso meta-learner
stacking_regressor = StackingCVRegressor(regressors=tuned_base_models,
                                         meta_regressor=Lasso(),
                                         cv=5,
                                         use_features_in_secondary=True,
                                         store_train_meta_features=True)

# Fit the stacking regressor
stacked_res = run_model(stacking_regressor)

# # Evaluate on test set
# y_pred = stacking_regressor.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error:", mse)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001175 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13604
[LightGBM] [Info] Number of data points in the train set: 876, number of used features: 76
[LightGBM] [Info] Start training from score 179921.786530
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13612
[LightGBM] [Info] Number of data points in the train set: 876, number of used features: 76
[LightGBM] [Info] Start training from score 181601.331050
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13614
[LightGBM] [Info] Number of data points in the train set: 876, number of used features: 76
[LightGBM] [Info] St

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.449e+11, tolerance: 6.648e+08
  model = cd_fast.enet_coordinate_descent(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000918 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13815
[LightGBM] [Info] Number of data points in the train set: 1095, number of used features: 77
[LightGBM] [Info] Start training from score 181712.286758


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


Model Performance For Traning Set
----------
r2_score:  0.9562
mean squared error:  265770235.2164
rmse:  29101.0144
----------
Model Performance For Test Set
----------
r2_score:  0.8791
mean squared error:  846869038.2686
rmse:  29101.0144
----------
StackingCVRegressor(meta_regressor=Lasso(),
                    regressors=[RandomForestRegressor(n_estimators=50),
                                GradientBoostingRegressor(n_estimators=50),
                                XGBRegressor(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_metric

In [53]:
stacked_res

{'Model Name': 'StackingCV',
 'r2_score': 0.8791,
 'mean squared error': 846869038.2686,
 'rmse': 29101.0144}

In [54]:
df2 = pd.DataFrame(stacked_res,index=[0])
df2.head()

,Model Name,r2_score,mean squared error,rmse
0,StackingCV,0.8791,8.468690e+08,29101.0144


In [55]:
df = pd.concat([df, df2], ignore_index=True)  # Combine row-wise

In [56]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [57]:
linear = run_model(lr)

Model Performance For Traning Set
----------
r2_score:  0.8115
mean squared error:  1144700982.8386
rmse:  35608.4913
----------
Model Performance For Test Set
----------
r2_score:  0.819
mean squared error:  1267964650.5746
rmse:  35608.4913
----------
LinearRegression()
__________


In [58]:
df3 = pd.DataFrame(linear,index=[0])
df3.head()

,Model Name,r2_score,mean squared error,rmse
0,LinearRegression(),0.819,1.267965e+09,35608.4913


In [59]:
df = pd.concat([df, df3], ignore_index=True)  # Combine row-wise

In [60]:
df

,Model Name,r2_score,mean squared error,rmse
0,RandomFore,0.8688,9.192270e+08,30318.7574
1,GradientBo,0.8737,8.847044e+08,29743.9806
2,XGBRegress,0.8681,9.236571e+08,30391.7283
3,LGBMRegressor(),0.8807,8.359335e+08,28912.5138
4,<catboost.,0.8868,7.930067e+08,28160.3740
5,StackingCV,0.8791,8.468690e+08,29101.0144
6,LinearRegression(),0.8190,1.267965e+09,35608.4913


In [61]:
df.to_csv("Min_Max_Scaling_Analysis.csv")

### Plot Graphs

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([random_forest,cat_boost,stack,stack_model])
compare_results

In [ ]:
def plot_graphs(results: Dict[str,List[float]]):
    
    loss = results['rmse']
    